# CS 20 : TensorFlow for Deep Learning Research
## Lecture 04 : Eager execution
You no longer need to worry about...

1. placeholders
2. sessions
3. control dependencies
4. "lazy loading"
5. {name, variable, op} scopes

### notice
#### Main changes

The API for Variables will then change in the following ways for TF 2.0:

*   ***tf.Variable will become an abstract base class with a well-defined interface and a scoped factory to construct instances***
    *   ***users will be able to implement their own variable-like objects by subclassing tf.Variable and adding a scoped factory function to use those variables***
*   ***variable_scope and get_variable will be removed***
    *   the tf 1.0 version of variable_scope and get_variable will be left in tf.compat.v1
    *   ***to control variable naming users can use tf.name_scope + tf.Variable***
    *   whether a variable is shared across sessions / processes will be controlled by a constructor argument to tf.Variable; no other type of scope reuse will be done in the framework
    *   scoped partitioning will be implemented as a factory function at first
    *   libraries and users are encouraged to reuse variables by reusing their objects, like Keras layers do
    *   custom_getters will have the following API: [variable_creator_scope](https://github.com/tensorflow/tensorflow/blob/567189980f7a1c2aa09a5170bd8d01a6ec37d303/tensorflow/python/ops/variable_scope.py#L2402)
*   the default implementation of the tf.Variable interface will be ResourceVariable
    *   RefVariable will be kept in tf.compat.v1 and will be the default implementation for tf.compat.v1.Variable
    *   tf.compat.v1.Variable will have a use_resource argument to control whether a resource variable or a ref variable will be created
*   symbols like tf.assign* will be removed in favor of methods in tf.Variable
    *   in tf.compat.v1 these symbols will be marked as deprecated and will call the corresponding methods in the Variable object instead
https://github.com/tensorflow/community/blob/master/rfcs/20180817-variables-20.md

### Setup

In [1]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()

### Boilerplate
```python
x = tf.placeholder(dtype = tf.float32, shape = [1, 1])
m = tf.matmul(x, x)

print(m)
with tf.Session() as sess:
    m_out = sess.run(m, feed_dict = {x : [[2.]]})
    print(m_out, m_out.shape)
```

```python
Tensor("mul:0", shape=(1, 1), dtype=float32)
[[4.]] (1, 1)
```

**When using `tf.enable_eager_execution()`, Bolierplate changes as belows**

In [2]:
x = [[2.]]
m = tf.matmul(x, x)
print(m) # No sessions()!
print(tf.get_default_graph().as_graph_def()) # No graphs!

tf.Tensor([[4.]], shape=(1, 1), dtype=float32)
versions {
  producer: 27
}



### Lazy Loading
Each iteration adds nodes to the graph

```python
x = tf.constant(value = [[1,2],[3,4]], dtype = tf.int32)

with tf.Session() as sess:
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            print(sess.run(x[i, j]))
```

```python
1
2
3
4
```

**When using `tf.enable_eager_execution()`, not graph**

In [3]:
x = tf.constant(value = [[1,2],[3,4]], dtype = tf.int32)

for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        print(x[i, j])

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


### Tensors act like numpy arrays

In [9]:
# Tensors are backed by NumPy arrays
# Tensors are compatible with NumPy functions
x = tf.constant(value = [[1.,2.,3]])
assert type(x.numpy()) == np.ndarray
squared = np.square(x)
print(squared)

# Tensors are iterable!
for i in x[0]:
    print(i)

[[1. 4. 9.]]
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(3.0, shape=(), dtype=float32)
